In [ ]:
# set parameters
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import scipy
import sys
sys.path.append('../../assets/python/')
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

COLORS = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [9]:
# functions for generating encounters
import numpy as np
from scipy.special import erf
import sys
sys.path.append('../../assets/python/')
import dmg5e
import dmg5e2024

def log_normal_mean(mu, sigma):
    return np.log( (mu**2)/np.sqrt(mu**2 + sigma**2) )

def log_normal_sigma(mu, sigma):
    return np.sqrt(np.log(1 + (sigma/mu)**2))

def log_normal_pdf(mu, sigma, x):
    return np.exp(-0.5*((np.log(x) - mu)/sigma)**2) / (x*sigma*np.sqrt(2*np.pi))

def normal_pdf(mu, sigma, x):
    return np.exp(-0.5*((x - mu)/sigma)**2) / np.sqrt(2*np.pi*sigma**2)

def normal_cdf(mu, sigma, x):
    return 0.5*(1 + erf( (x - mu) / np.sqrt(2*sigma**2) ))

def add_monster(pc_lvls, mon_crs, xp_budget, **kwargs):
    xp_mult_p1 = dmg5e.encounter_xp_multiplier(len(pc_lvls), len(mon_crs)+1)
    adj_xp_p1 = xp_mult_p1*dmg5e.encounter_xp(mon_crs)
    xp_rem = (xp_budget - adj_xp_p1)/xp_mult_p1

    cr_list = np.array(dmg5e.MONSTER_DEFAULTS['CR'])
    xp_list = np.array(dmg5e.MONSTER_DEFAULTS['XP'])
    cr_list = cr_list[(xp_list > 0) & (xp_list <= xp_rem)]
    xp_list = xp_list[(xp_list > 0) & (xp_list <= xp_rem)]

    if 'cr_range' in kwargs:
        indx = (cr_list <= max(kwargs['cr_range'])) & (cr_list >= min(kwargs['cr_range']))
        xp_list = xp_list[indx]
        cr_list = cr_list[indx]
    
    if len(cr_list) == 0: return None

    i = int(np.floor(np.random.uniform(0, len(cr_list))))
    return cr_list[i]


def build_encounter(pc_lvls, xp_range, **kwargs):
    enc_xp_max = max(xp_range)
    enc_xp_min = min(xp_range)
    mon_crs = []
    i = 0
    imax = 100
    while i < imax:
        i += 1
        mon_crs = []
        new_cr = add_monster(pc_lvls, mon_crs, enc_xp_max, **kwargs)
        while new_cr:
            mon_crs.append(new_cr)
            enc_adj_xp = dmg5e.encounter_adjusted_xp(pc_lvls, mon_crs)
            if enc_adj_xp >= enc_xp_min: break
            if kwargs.get('max_size', 1000) <= len(mon_crs): break
            new_cr = add_monster(pc_lvls, mon_crs, enc_xp_max, **kwargs)

        if not mon_crs:
            print('unable to create encounter')
            print(f'  {pc_lvls}, {xp_range}')
            break

        enc_adj_xp = dmg5e.encounter_adjusted_xp(pc_lvls, mon_crs)
        if enc_adj_xp >= enc_xp_min: break
    
    if i >= imax:
        print(f'build_encounter exceeded the maximum number of iterations!')
    
    return sorted(mon_crs, reverse=True)

def pc_hp_baseline( lvl ):
    return 1 + 7*lvl

def party_hp_baseline( *args ):
    if len(args) == 1:
        levels = args[0]
    else:
        levels = args
    return sum([pc_hp_baseline(lvl) for lvl in levels])

def monster_damage_cv( mon_cr ):
    # CR  1: 0.9/sqrt(2.5)
    # CR 30: 0.1/sqrt(2.5)
    return (0.93 - mon_cr*0.03)/np.sqrt(2.5)

def encounter_xp_sigma( pc_lvls, mon_crs ):
    #em = dmg5e.encounter_xp_multiplier(len(pc_lvls), len(mon_crs))
    xp_var = sum([(dmg5e.monster_default_stats(cr)['XP']*monster_damage_cv(cr))**2 for cr in mon_crs])
    return np.sqrt(xp_var)

def encounter_damage_cv( pc_lvls, mon_crs ):
    em = dmg5e.encounter_xp_multiplier(len(pc_lvls), len(mon_crs))
    xp_tot = em*dmg5e.encounter_xp(mon_crs)
    xp_var = em*sum([(dmg5e.monster_default_stats(cr)['XP']*monster_damage_cv(cr))**2 for cr in mon_crs])
    return np.sqrt(xp_var)/xp_tot

def party_encounter_hp_lost(pc_lvls, mon_crs):
    """Generates a random amount of damage done to the PCs during an encounter for testing.
    """
    #em = dmg5e.encounter_xp_multiplier(len(pc_lvls), len(mon_crs))
    party_level = sum(pc_lvls)/len(pc_lvls)
    aoe_pct = min(0.5, 0.15 + 0.05*party_level)
    #em = encounter_multiplier_approx(len(pc_lvls), len(mon_crs), 0, aoe_pct, 0.7)
    em = 1
    
    enc_adj_xp_mean = em*dmg5e.encounter_xp(mon_crs)
    enc_adj_xp_sigma = np.sqrt(em)*encounter_xp_sigma(pc_lvls, mon_crs)
    party_enc_max = dmg5e.party_xp_thresholds(pc_lvls)['Daily']/2
    party_hp = party_hp_baseline(pc_lvls)
    enc_xp_rand = max(0, np.random.normal(enc_adj_xp_mean, enc_adj_xp_sigma))
    return np.round((enc_xp_rand/party_enc_max)*party_hp, 0)

def encounter_multiplier_approx(pc_count=4, npc_count=1, pc_deaths=0, aoe_pct=0, aoe_eff=1.0):
    """Returns the approximate encounter multiplier
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    pc_deaths -- number of PCs expected to die during encounter
    aoe_pct -- decimal percent of damage done to each NPC using AoE effects
    AoE_eff -- ratio of AoE DPR per target to single target DPR
    """
    pd = min(pc_deaths, pc_count)
    em = (npc_count + 1)*(1 - aoe_pct)/2 + aoe_pct/aoe_eff
    em *= 8.0*pc_count/(2*pc_count*pc_count - 2*pd*pc_count + pd*(pd + 1))
    return em

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

In [10]:
# generate the simulated dataset
import pandas as pd

entries = 1000

print(f'Generating party information', end=' ... ')
# generate party levels
mu = 6
sigma = 5
ln_mu = log_normal_mean(mu, sigma)
ln_sigma = log_normal_sigma(mu, sigma)
#party_level_dist = [min(lvl, 20) for lvl in np.round(np.random.lognormal(ln_mu, ln_sigma, entries), 0)]
party_level_dist = np.round(np.random.lognormal(ln_mu, ln_sigma, 2*entries))
party_level_dist = party_level_dist[(party_level_dist >= 1) & (party_level_dist <= 20)]
party_level_dist = party_level_dist[0:entries]

# generate party size
m_party_size = 4
s_party_size = 1.5
#party_size_dist = [min(max(s, 2), 8) for s in np.round(np.random.normal(m_party_size, s_party_size, entries), 0)]
party_size_dist = np.round(np.random.normal(m_party_size, s_party_size, 2*entries))
party_size_dist = party_size_dist[(party_size_dist >= 2) & (party_size_dist <= 8)]
party_size_dist = party_size_dist[0:entries]

# construct parties
encounters = []
for l, n in zip(party_level_dist, party_size_dist):
    encounters.append({'pc_levels': [int(l)]*int(n)})
print('done.')

print(f'Constructing encounters', end=' ... ')
# determine XP ranges for each party
# next, need to generate random encounters ...
difficulties = ['Easy','Medium','Hard','Deadly']
diff_dist = [difficulties[int(i)] for i in np.floor(np.random.uniform(0, 4, entries))]
for e, d in zip(encounters, diff_dist):
    e['xp_range'] = dmg5e.party_xp_ranges(e['pc_levels'])[d]
    e['cr_range'] = [min(e['pc_levels'])/6, 30]
    e['mon_crs'] = build_encounter(e['pc_levels'], e['xp_range'], cr_range=e['cr_range'], max_size=20)
    e['hp_lost'] = max(1, party_encounter_hp_lost(e['pc_levels'], e['mon_crs']))
print('done.')

data = {
    'pc_levels': [e['pc_levels'] for e in encounters],
    'mon_crs': [e['mon_crs'] for e in encounters],
    'hp_lost': [e['hp_lost'] for e in encounters],
}
dfe = pd.DataFrame.from_dict(data)
dfe['party_level'] = dfe.apply(lambda r: sum(r['pc_levels'])/len(r['pc_levels']), axis=1)
dfe['party_size'] = dfe.apply(lambda r: len(r['pc_levels']), axis=1)
dfe['encounter_size'] = dfe.apply(lambda r: len(r['mon_crs']), axis=1)
#dfe['encounter_cr'] = dfe.apply(lambda r: np.round(sum(r['mon_crs'])/len(r['mon_crs']), 0), axis=1)
dfe['encounter_cr'] = dfe.apply(lambda r: sum(r['mon_crs'])/len(r['mon_crs']), axis=1)
dfe['party_hp'] = dfe.apply(lambda r: party_hp_baseline(r['pc_levels']), axis=1)
dfe['hp_lost_ratio'] = dfe['hp_lost']/dfe['party_hp']
dfe['pc_hp_lost'] = dfe.apply(lambda r: r['hp_lost']/len(r['pc_levels']), axis=1)

dfe['2014 party xp'] = dfe.apply(lambda r: dmg5e.party_xp_thresholds(r['pc_levels'])['Daily']/2, axis=1)
dfe['2014 difficulty'] = dfe.apply(lambda r: dmg5e.encounter_difficulty(r['pc_levels'], r['mon_crs']), axis=1)
dfe['2014 xp'] = dfe.apply(lambda r: dmg5e.encounter_adjusted_xp(r['pc_levels'], r['mon_crs']), axis=1)
dfe['2014 xp ratio'] = dfe['2014 xp']/dfe['2014 party xp']
dfe['2014 accuracy'] = dfe['hp_lost_ratio']/dfe['2014 xp ratio']

dfe['2024 party xp'] = dfe.apply(lambda r: dmg5e2024.party_xp_thresholds(r['pc_levels'])['Daily']/2, axis=1)
dfe['2024 difficulty'] = dfe.apply(lambda r: dmg5e2024.encounter_difficulty(r['pc_levels'], r['mon_crs']), axis=1)
dfe['2024 xp'] = dfe.apply(lambda r: dmg5e2024.encounter_adjusted_xp(r['pc_levels'], r['mon_crs']), axis=1)
dfe['2024 xp ratio'] = dfe['2024 xp']/dfe['2024 party xp']
dfe['2024 accuracy'] = dfe['hp_lost_ratio']/dfe['2024 xp ratio']

dfe = dfe.astype({'2014 difficulty': 'category', '2024 difficulty': 'category'})
dfe['2014 difficulty'] = dfe['2014 difficulty'].cat.set_categories(['Trivial','Easy','Medium','Hard','Deadly'], ordered=True)
dfe['2024 difficulty'] = dfe['2024 difficulty'].cat.set_categories(['Low','Moderate','High','Very High'], ordered=True)
dfe

Generating party information ... done.
Constructing encounters ... done.


,pc_levels,mon_crs,hp_lost,party_level,party_size,encounter_size,encounter_cr,party_hp,hp_lost_ratio,pc_hp_lost,2014 party xp,2014 difficulty,2014 xp,2014 xp ratio,2014 accuracy,2024 party xp,2024 difficulty,2024 xp,2024 xp ratio,2024 accuracy
0,"[3, 3, 3]","[1.0, 1.0, 1.0]",20.0,3.0,3,3,1.0,66,0.303030,6.666667,1800.0,Deadly,1200.0,0.666667,0.454545,1800.0,Moderate,600.0,0.333333,0.909091
1,"[4, 4, 4, 4]","[4.0, 1.0]",72.0,4.0,4,2,2.5,116,0.620690,18.000000,3400.0,Hard,1950.0,0.573529,1.082228,3400.0,Moderate,1300.0,0.382353,1.623342
2,"[3, 3, 3, 3, 3]","[1.0, 1.0]",19.0,3.0,5,2,1.0,110,0.172727,3.800000,3000.0,Easy,600.0,0.200000,0.863636,3000.0,Low,400.0,0.133333,1.295455
3,"[2, 2, 2, 2, 2]","[3.0, 1.0]",50.0,2.0,5,2,2.0,75,0.666667,10.000000,1500.0,Deadly,1350.0,0.900000,0.740741,1500.0,High,900.0,0.600000,1.111111
4,"[2, 2]","[0.5, 0.5]",11.0,2.0,2,2,0.5,30,0.366667,5.500000,600.0,Deadly,400.0,0.666667,0.550000,600.0,Low,200.0,0.333333,1.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"[5, 5, 5, 5, 5]","[6.0, 4.0]",94.0,5.0,5,2,5.0,180,0.522222,18.800000,8750.0,Hard,5100.0,0.582857,0.895969,8750.0,Moderate,3400.0,0.388571,1.343954
996,"[3, 3, 3, 3, 3, 3, 3]","[4.0, 2.0]",38.0,3.0,7,2,3.0,154,0.246753,5.428571,4200.0,Medium,1550.0,0.369048,0.668622,4200.0,Moderate,1550.0,0.369048,0.668622
997,"[2, 2, 2]","[0.5, 0.5]",16.0,2.0,3,2,0.5,45,0.355556,5.333333,900.0,Medium,300.0,0.333333,1.066667,900.0,Low,200.0,0.222222,1.600000
998,"[5, 5, 5, 5, 5, 5]","[9.0, 3.0]",175.0,5.0,6,2,6.0,216,0.810185,29.166667,10500.0,Hard,5700.0,0.542857,1.492446,10500.0,High,5700.0,0.542857,1.492446


In [11]:
# Fig. 1: plots the distribution of encounters by party size
fig = go.Figure()

# simulated data
name = 'simulated data'
dfG = dfe[['party_size','hp_lost']].groupby('party_size').count().reset_index()
fig.add_trace(go.Bar(
    x=dfG['party_size'],
    y=dfG['hp_lost'],
    name=name,
    hovertemplate=
        f'<b>{name}</b><br>'+
        'Party size %{x:.0f}<br>'+
        'Encounters %{y:.0f}'+
        '<extra></extra>'
))

# theoretical distribution
mu = 4
sigma = 1.5
x = np.linspace(2, 8, 100)
y = normal_pdf(mu, sigma, x)*sum(dfG['hp_lost'])
name = 'theoretical distribution'
fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='lines', 
    name=name,
    hoverinfo='skip',
))

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='PCs per encounter',
        autorange=True,
        tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'encounters',
        autorange=True,
        tickmode='auto',
        minor_tickmode='auto',
    ),
    legend=dict(
        xanchor='right', yanchor='top',
        x=1.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sim-party-size-dist-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sim-party-size-dist-small')

In [12]:
# Fig. 2: plots the distribution of encounters by party level
fig = go.Figure()

# simulated data
name = 'simulated data'
dfG = dfe[['party_level','hp_lost']].groupby('party_level').count().reset_index()
fig.add_trace(go.Bar(
    x=dfG['party_level'],
    y=dfG['hp_lost'],
    name=name,
    hovertemplate=
        f'<b>{name}</b><br>'+
        'Level %{x:.0f}<br>'+
        'Encounters %{y:.0f}'+
        '<extra></extra>'
))

# theoretical distribution
mu = 6
sigma = 5
ln_mu = log_normal_mean(mu, sigma)
ln_sigma = log_normal_sigma(mu, sigma)
x = np.linspace(1, 20, 200)
y = log_normal_pdf(ln_mu, ln_sigma, x)
name = 'theoretical distribution'
fig.add_trace(go.Scatter(
    x=x, 
    y=y*sum(dfG['hp_lost']),
    mode='lines', 
    name=name,
    hoverinfo='skip',
))

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='party level',
        range=[0,21],
        tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'encounters',
        autorange=True,
        tickmode='auto',
        minor_tickmode='auto',
    ),
    legend=dict(
        xanchor='right', yanchor='top',
        x=1.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600,
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sim-party-level-dist-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sim-party-level-dist-small')

In [13]:
# Fig. 3: plots the distribution of encounters by the number of monsters
fig = go.Figure()

# simulated data
name = 'simulated data'
dfG = dfe[['encounter_size','hp_lost']].groupby('encounter_size').count().reset_index()
fig.add_trace(go.Bar(
    x=dfG['encounter_size'],
    y=dfG['hp_lost'],
    name=name,
    hovertemplate=
        'Monsters %{x:.0f}<br>'+
        'Encounters %{y:.0f}'+
        '<extra></extra>'
))

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='monsters per encounter',
        autorange=True,
        tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'encounters',
        autorange=True,
        tickmode='auto',
        minor_tickmode='auto',
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sim-encounter-size-dist-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sim-encounter-size-dist-small')

In [14]:
# Fig. 4: plots the average monster CR faced by parties of each level
fig = go.Figure()

# plot monster data
dfG = dfe[['party_level','encounter_cr']].groupby(['party_level'], observed=False).mean().reset_index()
tfb.plot_data_and_fit(fig, 
    x=dfG['party_level'], 
    y=dfG['encounter_cr'], 
    line_color=COLORS[0],
    showlegend=False,
    hoverinfo='skip',
    print_coefficients=True,
)

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='party level',
        autorange=True,
        tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'average monster challenge rating',
        #range=[0,1.5],
        autorange=True,
        tickmode='auto',
    ),
    legend=dict(
        xanchor='left', yanchor='top',
        x=0.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600, 
    height=450
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sim-average-cr-vs-level-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sim-average-cr-vs-level-small')

CR >= 1: 0.89 + 0.56*CR


In [15]:
# Fig. 5: plots simulated damage ratio vs adjusted XP ratio
fig = go.Figure()

# plot monster data
df1 = dfe[dfe['encounter_size'].ge(1)]
text = df1.apply(lambda r: f"XP ratio {r['2014 xp ratio']:.3f}<br>accuracy {r['2014 accuracy']:.3f}<br>"
                 f"Difficulty {r['2014 difficulty']}<br>PCs {r['pc_levels']}<br>CRs {r['mon_crs']}", axis=1)
fig.add_box(
    y=df1['2014 accuracy'], 
    x=['2014 rules']*len(df1),
    boxpoints='all',
    width=0.4,
    pointpos=-2,
    whiskerwidth=0.4,
    jitter=0.7,
    marker_size=3,
    marker_line_width=0,
    line_color=COLORS[0],
    name=f'2014 rules',
    legendgroup=f'2014 rules',
    text=text,
    hoveron='points',
    hovertemplate='%{text}<extra></extra>',
)

text = df1.apply(lambda r: f"XP ratio {r['2014 xp ratio']:.3f}<br>accuracy {r['2014 accuracy']:.3f}<br>"
                 f"Difficulty {r['2014 difficulty']}<br>PCs {r['pc_levels']}<br>CRs {r['mon_crs']}", axis=1)
fig.add_box(
    y=df1['2024 accuracy'], 
    x=['2024 rules']*len(df1),
    boxpoints='all',
    width=0.4,
    pointpos=-2,
    whiskerwidth=0.4,
    jitter=0.7,
    marker_size=3,
    marker_line_width=0,
    line_color=COLORS[1],
    name=f'2024 rules',
    legendgroup=f'2024 rules',
    text=text,
    hoveron='points',
    hovertemplate='%{text}<extra></extra>',
)
print('2014', np.mean(df1['2014 accuracy']))
print('2024', np.mean(df1['2024 accuracy']))

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        autorange=True,
        tickmode='auto',
        minor_tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'accuracy',
        range=[0,2.7],
        tickmode='auto',
        tickformat='.1f',
        minor_tickmode='auto',
    ),
    legend=dict(
        xanchor='right', yanchor='top',
        x=1.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.write_image('../../assets/images/calibrating-encounter-math.png')
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sim-accuracy-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sim-accuracy-distribution-small')

2014 0.754170658391639
2024 0.9845960191015826


In [16]:
# Fig. 6: shows the encounter XP accuracy as a function of encounter size
fig = go.Figure()

# plot monster data
dft = dfe[dfe['encounter_size'].between(1,5)]
dfg = dft[['encounter_size','2014 accuracy','2024 accuracy']].groupby(['encounter_size'], observed=False).mean().reset_index()

# 2014 rules
name = '2014 rules'
color = COLORS[0]
fig.add_scatter(
    x=tfb.jitter(dft['encounter_size']+0.1, 0.1), 
    y=dft['2014 accuracy'], 
    text=dft.apply(lambda r: f"XP ratio {r['2014 xp ratio']:.3f}<br>accuracy {r['2014 accuracy']:.3f}<br>"
                 f"Difficulty {r['2014 difficulty']}<br>PCs {r['pc_levels']}<br>CRs {r['mon_crs']}", axis=1),
    mode='markers',
    showlegend=False,
    name=name,
    legendgroup=name,
    opacity=0.5,
    marker_size=3,
    marker_line_width=0,
    line_color=color,
    hovertemplate='%{text}<extra></extra>',
)
fig.add_scatter(
    x=dfg['encounter_size'], 
    y=dfg['2014 accuracy'], 
    mode='lines',
    name=name,
    legendgroup=name,
    line_color=color,
    hoverinfo='skip',
)


# 2024 rules
name = '2024 rules'
color = COLORS[1]
fig.add_scatter(
    x=tfb.jitter(dft['encounter_size']-0.1, 0.1), 
    y=dft['2024 accuracy'], 
    text=dft.apply(lambda r: f"XP ratio {r['2014 xp ratio']:.3f}<br>accuracy {r['2014 accuracy']:.3f}<br>"
                 f"Difficulty {r['2014 difficulty']}<br>PCs {r['pc_levels']}<br>CRs {r['mon_crs']}", axis=1),
    mode='markers',
    showlegend=False,
    name=name,
    legendgroup=name,
    opacity=0.5,
    marker_size=3,
    marker_line_width=0,
    line_color=color,
    hovertemplate='%{text}<extra></extra>',
)
fig.add_scatter(
    x=dfg['encounter_size'], 
    y=dfg['2024 accuracy'], 
    mode='lines',
    name=name,
    legendgroup=name,
    line_color=color,
    hoverinfo='skip',
)

x = [1,2,3,4,5]
fig.add_scatter(
    x=x, 
    y=[1/dmg5e.encounter_xp_multiplier(4, v) for v in x], 
    mode='lines',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
)


# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='monsters per encounter',
        range=[0.1,5.9],
        tick0=0, dtick=1,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text=f'accuracy',
        range=[0,2],
        tickmode='auto',
        minor_tickmode='auto',
        tickformat='.1f',
    ),
    legend=dict(
        xanchor='right', yanchor='top',
        x=1.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sim-accuracy-vs-encounter-size-large', marker_size=3, line_width=3)
    tfb.save_fig_html(fig, format='small', name=f'./fig-sim-accuracy-vs-encounter-size-small', marker_size=2, line_width=2)

In [17]:
# Fig. 7: shows the encounter XP accuracy as a function of party size
fig = go.Figure()

# plot monster data
dft = dfe[dfe['encounter_size'].between(1,8)]
dfg = dft[['party_size','2014 accuracy','2024 accuracy']].groupby(['party_size'], observed=False).mean().reset_index()

# 2014 rules
name = '2014 rules'
color = COLORS[0]
fig.add_scatter(
    x=tfb.jitter(dft['party_size']+0.1, 0.1), 
    y=dft['2014 accuracy'], 
    text=dft.apply(lambda r: f"XP ratio {r['2014 xp ratio']:.3f}<br>accuracy {r['2014 accuracy']:.3f}<br>"
                 f"Difficulty {r['2014 difficulty']}<br>PCs {r['pc_levels']}<br>CRs {r['mon_crs']}", axis=1),
    mode='markers',
    showlegend=False,
    name=name,
    legendgroup=name,
    opacity=0.5,
    marker_size=3,
    marker_line_width=0,
    line_color=color,
    hovertemplate='%{text}<extra></extra>',
)
fig.add_scatter(
    x=dfg['party_size'], 
    y=dfg['2014 accuracy'], 
    mode='lines',
    name=name,
    legendgroup=name,
    line_color=color,
    hoverinfo='skip',
)


# 2024 rules
name = '2024 rules'
color = COLORS[1]
fig.add_scatter(
    x=tfb.jitter(dft['party_size']-0.1, 0.1), 
    y=dft['2024 accuracy'], 
    text=dft.apply(lambda r: f"XP ratio {r['2014 xp ratio']:.3f}<br>accuracy {r['2014 accuracy']:.3f}<br>"
                 f"Difficulty {r['2014 difficulty']}<br>PCs {r['pc_levels']}<br>CRs {r['mon_crs']}", axis=1),
    mode='markers',
    showlegend=False,
    name=name,
    legendgroup=name,
    opacity=0.5,
    marker_size=3,
    marker_line_width=0,
    line_color=color,
    hovertemplate='%{text}<extra></extra>',
)
fig.add_scatter(
    x=dfg['party_size'], 
    y=dfg['2024 accuracy'], 
    mode='lines',
    name=name,
    legendgroup=name,
    line_color=color,
    hoverinfo='skip',
)


x = [2,3,4,5,6,7,8]
fig.add_scatter(
    x=x, 
    y=[1/dmg5e.encounter_xp_multiplier(v, 2) for v in x], 
    mode='lines',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
)

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='PCs per encounter',
        range=[1.1,8.9],
        tick0=0, dtick=1,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text=f'accuracy',
        range=[0,2],
        tickmode='auto',
        minor_tickmode='auto',
        tickformat='.1f',
    ),
    legend=dict(
        xanchor='right', yanchor='top',
        x=1.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sim-accuracy-vs-party-size-large', marker_size=3, line_width=3)
    tfb.save_fig_html(fig, format='small', name=f'./fig-sim-accuracy-vs-party-size-small', marker_size=2, line_width=2)

# Unused

In [18]:
# plots the distribution of encounters by difficulty
# create figure
fig = go.Figure()

dfG = dfe[['2014 difficulty','hp_lost']].groupby('2014 difficulty', observed=True).count().reset_index()
fig.add_trace(go.Bar(
    x=dfG['2014 difficulty'],
    y=dfG['hp_lost'],
    hoverinfo='skip',
))

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='difficulty',
        autorange=True,
        tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'encounters',
        autorange=True,
        tickmode='auto',
        minor_tickmode='auto',
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-ab-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-ab-vs-cr-small')

In [19]:
# shows the average HP lost per PC as a function of party level for each encounter difficulty
fig = go.Figure()

# plot monster data
i = 0
for diff in ['Easy','Medium','Hard','Deadly']:
    i += 1
    dft = dfe[dfe['2014 difficulty'].eq(diff) & dfe['encounter_size'].ge(2)]
    dfg = dft[['party_level','pc_hp_lost']].groupby(['party_level'], observed=False).mean().reset_index()
    tfb.plot_data_and_fit(fig, 
        x=dfg['party_level'], 
        y=dfg['pc_hp_lost'], 
        line_color=COLORS[i],
        name=f'{diff}',
        legendgroup=f'{diff}',
        hoverinfo='skip',
        print_coefficients=True,
    )

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='average party level',
        range=[0,21],
        tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'HP lost per PC',
        range=[0,80],
        tickmode='auto',
        minor_tickmode='auto',
    ),
    legend=dict(
        xanchor='left', yanchor='top',
        x=0.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-ab-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-ab-vs-cr-small')

CR >= 1: 0.95 + 0.98*CR
CR >= 1: -1.88 + 2.07*CR
CR >= 1: -0.39 + 2.57*CR
CR >= 1: 2.52 + 3.08*CR


In [20]:
# shows the average HP lost ratio as a function of party level for each encounter difficulty
fig = go.Figure()

# plot monster data
bins = [0,0.15,0.30,0.45,0.7,1.0]
i = 0
for diff in ['Easy','Medium','Hard','Deadly']:
    i += 1
    dft = dfe[dfe['2014 difficulty'].eq(diff) & dfe['encounter_size'].ge(2)]
    dfg = dft[['party_level','hp_lost_ratio']].groupby(['party_level'], observed=False).median().reset_index()
    tfb.plot_data_and_fit(fig, 
        x=dfg['party_level'], 
        y=dfg['hp_lost_ratio'], 
        line_color=COLORS[i],
        name=f'{diff}',
        legendgroup=f'{diff}',
        hoverinfo='skip',
        print_coefficients=True,
    )

# show figure
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=70, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='average party level',
        range=[0,20],
        tickmode='auto',
    ),
    yaxis=dict(
        title_text=f'HP lost ratio',
        range=[0,1],
        tickmode='auto',
    ),
    legend=dict(
        xanchor='left', yanchor='top',
        x=0.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    ),
    width=600, 
    height=450,
)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-ab-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-ab-vs-cr-small')

CR >= 1: 0.16 + -0.00*CR
CR >= 1: 0.19 + 0.01*CR
CR >= 1: 0.32 + 0.00*CR
CR >= 1: 0.47 + -0.00*CR


In [21]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

#yCol = 'hp_lost_ratio'
#xCols = ['party_level','adj_xp_ratio','party_size','encounter_size']
yCol = '2014 xp ratio'
xCols = ['party_level','hp_lost_ratio','party_size','encounter_size']
aCols = [yCol] + xCols
df1 = dfe[aCols]
X = pd.get_dummies(data=df1[xCols], drop_first=False)
Y = df1[yCol]

model = LinearRegression()
model.fit(X, Y)

coeff_parameter = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coeff_parameter)

# generate report
X_Sm = sm.add_constant(X)
ls = sm.OLS(Y, X_Sm.astype(float)).fit()
print(ls.summary())

                Coefficient
party_level       -0.000959
hp_lost_ratio      0.560773
party_size        -0.041043
encounter_size     0.111376
                            OLS Regression Results                            
Dep. Variable:          2014 xp ratio   R-squared:                       0.486
Model:                            OLS   Adj. R-squared:                  0.484
Method:                 Least Squares   F-statistic:                     235.0
Date:                Tue, 08 Apr 2025   Prob (F-statistic):          4.97e-142
Time:                        23:35:37   Log-Likelihood:                 322.02
No. Observations:                1000   AIC:                            -634.0
Df Residuals:                     995   BIC:                            -609.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025

In [22]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

yCol = 'hp_lost_ratio'
xCols = ['party_level','party_size','encounter_size','2024 xp ratio']
#yCol = 'adj_xp_ratio'
#xCols = ['party_level','hp_lost_ratio','party_size','encounter_size']
aCols = [yCol] + xCols
df1 = dfe[aCols]
X = pd.get_dummies(data=df1[xCols], drop_first=False)
Y = df1[yCol]

model = LinearRegression()
model.fit(X, Y)

coeff_parameter = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coeff_parameter)

# generate report
X_Sm = sm.add_constant(X)
ls = sm.OLS(Y, X_Sm.astype(float)).fit()
print(ls.summary())

                Coefficient
party_level       -0.001020
party_size         0.006101
encounter_size     0.003214
2024 xp ratio      1.060723
                            OLS Regression Results                            
Dep. Variable:          hp_lost_ratio   R-squared:                       0.633
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     428.3
Date:                Tue, 08 Apr 2025   Prob (F-statistic):          1.45e-214
Time:                        23:35:37   Log-Likelihood:                 321.08
No. Observations:                1000   AIC:                            -632.2
Df Residuals:                     995   BIC:                            -607.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025